Code to read and write down Hough maxima from data files

In [ ]:
#! pip install particle==0.25.4

In [ ]:
import uproot
import pandas as pd
import numpy as np
#from scipy.ndimage import maximum_filter, label, find_objects
from skimage.feature import peak_local_max
from scipy.ndimage import gaussian_filter
from numpy.lib.stride_tricks import sliding_window_view

import os
import gc  #garbage collector
from pathlib import Path

#from particle import Particle
import hepunits.units as units
from collections import defaultdict

import matplotlib.pyplot as plt
import matplotlib.patches as patches

Define binning of the Hough accumulator

In [ ]:
nbin_phi, nbin_qpt   = 7000, 216

# Size (half-width) of the square
size = 16

# Peak finding
threshold_abs=5     # absolute threshold
#threshold_abs=5     # absolute threshold
threshold_rel=0.0  # peaks > 70% of max
min_distance=2      # at least 6 bins apart
smooth_sigma=0      # smooth before detection

tol = 6             # tolerance between reco and true peak

#slice_list = list(range(-1, 33))
#slice_list = list(range(12, 20))
slice_list = [-1]

num_files = 8  # number of files to be processed 

In [ ]:
#path = "/eos/user/t/tbold/EFTracking/HoughML/ttbar_pu10_insquare"   # directory with ttbar data files
#path = "/eos/user/t/tbold/EFTracking/HoughML/ttbar_pu10_insquare_full" 

#path = "/eos/user/t/tbold/EFTracking/HoughML/ttbar_pu100_insquare"
#path = "/eos/user/t/tbold/EFTracking/HoughML/ttbar_pu100_insquare_full"

path = "/eos/user/t/tbold/EFTracking/HoughML/pg_2mu_pu100_insquare"

Read simulation file with true tracks information

In [ ]:
root_files = sorted(Path(path).glob("particles*.root"))

print(f"Found {len(root_files)} ROOT files in {path}\n")

# --- LOOP THROUGH FILES AND LIST OBJECTS ---
for file in root_files:
    print(f"📁 {file.name}")
    tree = uproot.open(file)["particles"]

    # Get data as a dictionary of NumPy arrays
    arrays = tree.arrays(library="np")

# Convert manually into a DataFrame
df = pd.DataFrame(arrays)

print("Column names:")
print(list(df.columns))

df.head()

Get charges of all particles in event

In [ ]:
# Extended particle charge dictionary
PARTICLE_CHARGES = {
    # Gauge bosons
    22: 0,       # photon
    23: 0,       # Z boson
    24: 1,       # W+
    -24: -1,     # W-
    21: 0,       # gluon
    
    # Leptons
    11: -1,      # e-
    -11: 1,      # e+
    12: 0,       # ν_e
    -12: 0,      # ν_e bar
    13: -1,      # μ-
    -13: 1,      # μ+
    14: 0,       # ν_μ
    -14: 0,      # ν_μ bar
    15: -1,      # τ-
    -15: 1,      # τ+
    16: 0,       # ν_τ
    -16: 0,      # ν_τ bar
    
    # Quarks
    1: -1/3,     # d
    -1: 1/3,     # d bar
    2: 2/3,      # u
    -2: -2/3,    # u bar
    3: -1/3,     # s
    -3: 1/3,     # s bar
    4: 2/3,      # c
    -4: -2/3,    # c bar
    5: -1/3,     # b
    -5: 1/3,     # b bar
    6: 2/3,      # t
    -6: -2/3,    # t bar
    
    # Light mesons
    111: 0,      # π⁰
    211: 1,      # π+
    -211: -1,    # π-
    113: 0,      # ρ⁰
    213: 1,      # ρ+
    -213: -1,    # ρ-
    221: 0,      # η
    331: 0,      # η'
    130: 0,      # K_L⁰
    310: 0,      # K_S⁰
    311: 0,      # K⁰
    -311: 0,     # K⁰ bar
    321: 1,      # K+
    -321: -1,    # K-
    
    # Charmed mesons
    411: 1,      # D+
    -411: -1,    # D-
    421: 0,      # D⁰
    -421: 0,     # D⁰ bar
    
    # Bottom mesons
    511: 0,      # B⁰
    -511: 0,     # B⁰ bar
    521: 1,      # B+
    -521: -1,    # B-
    
    # Baryons
    2212: 1,     # proton
    -2212: -1,   # antiproton
    2112: 0,     # neutron
    -2112: 0,    # antineutron
    3122: 0,     # Λ
    -3122: 0,    # Λ bar
    3222: 1,     # Σ+
    -3222: -1,   # Σ+ bar
    3212: 0,     # Σ⁰
    -3212: 0,    # Σ⁰ bar
    3112: -1,    # Σ-
    -3112: 1,    # Σ- bar
    3312: -1,    # Ξ⁻
    -3312: 1,    # Ξ⁻ bar
    3322: 0,     # Ξ⁰
    -3322: 0,    # Ξ⁰ bar
}

def get_charge_from_pdg(pdg_id):
    """Get electric charge from PDG ID"""
    if pdg_id in PARTICLE_CHARGES:
        return PARTICLE_CHARGES[pdg_id]
    else:
        # Try to handle antiparticles automatically
        if pdg_id < 0 and -pdg_id in PARTICLE_CHARGES:
            return -PARTICLE_CHARGES[-pdg_id]
        return None

# Alternative: return 0 for unknown particles instead of None
def get_charge_safe(pdg_id):
    """Get charge, return 0 for unknown particles"""
    charge = get_charge_from_pdg(pdg_id)
    return charge if charge is not None else 0

In [ ]:

def get_charges(particle_ids):
    """
    Return a list of charges for a list of PDG IDs.
    If an ID is not known in the PDG database, returns None for that entry.
    """
    charges = []
    for pid in particle_ids:
        try:
            #p = Particle.from_pdgid(pid)
            #charges.append(int(p.charge))  # convert to int (e units)
            charges.append(get_charge_from_pdg(pid))
        except Exception:
            charges.append(None)  # unknown ID
    return charges


Get positions of true tracks on the Hough accumulator

In [ ]:
'''
def true_tracks(df, event_id):
    
    tracks = []
    
    for row in df.itertuples(index=False):
        if row.event_id == event_id: 
            phi = row.phi
            phi_bin = (phi+np.pi)*nbin_phi/(2.*np.pi)
            charges = get_charges(row.particle_type)
            #print("charges ",charges)
            
            
            # mask when charges != 0
            mask = (np.array(charges) != 0) & (np.array(row.number_of_hits) > 4)
            

            curv = charges/row.pt
            curv_bin = int(nbin_qpt/2.)+curv*int(nbin_qpt/2.)
            #print(type(curv_bin))
            accu = np.stack((np.array(phi_bin)[mask], np.array(curv_bin)[mask], 
                             np.array(row.eta)[mask], np.array(row.vz)[mask], np.array(row.number_of_hits)[mask],
                             np.array(row.pz/row.pt)[mask], np.array(row.particle_type)[mask]), axis=1)
            
            #print(accu.shape, len(phi))

            return accu 
'''

In [ ]:

def true_tracks(df, event_id):
    
    result_dfs = []
    
    for row in df.itertuples(index=False):
        if row.event_id == event_id: 
            phi = row.phi
            phi_bin = (phi+np.pi)*nbin_phi/(2.*np.pi)
            charges = get_charges(row.particle_type)
            
            # mask when charges != 0
            mask = (np.array(charges) != 0) & (np.array(row.number_of_hits) > 4)
            
            curv = charges/row.pt
            curv_bin = int(nbin_qpt/2.)+curv*int(nbin_qpt/2.)
            
            # Apply mask to ALL arrays that need filtering
            filtered_phi_bin = np.array(phi_bin)[mask]
            filtered_curv_bin = np.array(curv_bin)[mask]
            filtered_eta = np.array(row.eta)[mask]
            filtered_vz = np.array(row.vz)[mask]
            filtered_number_of_hits = np.array(row.number_of_hits)[mask]
            filtered_pz_over_pt = np.array(row.pz/row.pt)[mask]
            filtered_particle_type = np.array(row.particle_type)[mask]
            
            # Also apply mask to other fields that should be filtered
            filtered_phi = np.array(row.phi)[mask] if hasattr(row.phi, '__len__') else np.full(np.sum(mask), row.phi)
            filtered_pt = np.array(row.pt)[mask] if hasattr(row.pt, '__len__') else np.full(np.sum(mask), row.pt)
            filtered_pz = np.array(row.pz)[mask] if hasattr(row.pz, '__len__') else np.full(np.sum(mask), row.pz)
            filtered_event_id = np.full(np.sum(mask), row.event_id)
            
            # Add 'reco' column with all zeros
            filtered_reco = np.zeros(np.sum(mask))
            
            # Create DataFrame with ALL filtered fields
            accu_df = pd.DataFrame({
                'phi_bin': filtered_phi_bin,
                'curv_bin': filtered_curv_bin,
                'eta': filtered_eta,
                'vz': filtered_vz,
                'number_of_hits': filtered_number_of_hits,
                'pz_over_pt': filtered_pz_over_pt,
                'particle_type': filtered_particle_type,
                # Add additional fields with proper masking
                'event_id': filtered_event_id,
                'phi': filtered_phi,
                'pt': filtered_pt,
                'pz': filtered_pz,
                'reco': filtered_reco,
            })
            
#            # Add any other fields that exist in the original row (with proper masking)
#            for field in row._fields:
#                if field not in accu_df.columns and field not in ['Index', 'index']:
#                    original_value = getattr(row, field)
#                    # Apply mask if it's an array, otherwise broadcast
#                    if hasattr(original_value, '__len__') and len(original_value) == len(mask):
#                        filtered_value = np.array(original_value)[mask]
#                    else:
#                        filtered_value = np.full(np.sum(mask), original_value)
#                    accu_df[field] = filtered_value
#            
            result_dfs.append(accu_df)
    
    # Combine all DataFrames if multiple rows match the event_id
    if result_dfs:
        return pd.concat(result_dfs, ignore_index=True)
    else:
        return pd.DataFrame()


Create true track dictionary

In [ ]:
'''
# Initialize dictionary that automatically creates empty lists for new keys
truetracks = defaultdict(list)

# Your loop
for event_id in df["event_id"]:
    #print("Event: ",event_id)
    truetracks[event_id].append(true_tracks(df, event_id))
    #print(truetracks[event_id])

# Convert back to regular dict if needed
truetracks = dict(truetracks)
#print(truetracks)
'''

# Initialize as regular dictionary
truetracks = {}

# Your loop - assign directly instead of appending
for event_id in df["event_id"]:
    #print("Event: ",event_id)
    truetracks[event_id] = true_tracks(df, event_id)  # Direct assignment, not append
    #print(truetracks[event_id])

# No need to convert back since it's already a regular dict
#print(truetracks)

In [ ]:
#for key, value in truetracks.items():
#    print(f"{key}: {type(value)} - {len(value)}")

Vectorized sliding window to find maxima

In [ ]:

def vectorized_2d_sliding_peaks(matrix, min_height=6, window_size=5):
    """
    Fully vectorized sliding window peak detection for 2D arrays
    """
    if window_size % 2 == 0:
        window_size += 1  # Ensure odd window size
    
    half_window = window_size // 2
    peaks = []
    
    # Create 2D sliding window view - this is the key optimization
    # Shape: (rows, cols, window_size, window_size)
    windows_2d = sliding_window_view(matrix, (window_size, window_size))
    
    # Center position in each window
    center_r, center_c = half_window, half_window
    
    # Extract center values and neighbors
    center_values = windows_2d[:, :, center_r, center_c]
    
    # Vectorized peak detection conditions
    is_peak = np.ones_like(center_values, dtype=bool)
 


    # Check if center is maximum in the window
    window_maxima = np.max(windows_2d, axis=(2, 3))
    is_peak &= (center_values == window_maxima)
    
    # Check minimum height
    is_peak &= (center_values >= min_height)
    
    # Get coordinates of peaks
    peak_rows, peak_cols = np.where(is_peak)
    
    # Adjust coordinates (sliding window shifts indices)
    adjusted_rows = peak_rows + half_window
    adjusted_cols = peak_cols + half_window
    
    
    #return np.stack((adjusted_rows, adjusted_cols),axis=1)
    
    
    # Merge close peaks
    if len(adjusted_rows) > 0:
        # Create array of peak coordinates and values
        peak_coords = np.stack((adjusted_rows, adjusted_cols), axis=1)
        peak_values = matrix[adjusted_rows, adjusted_cols]
        
        # Calculate distance matrix between all peaks
        from scipy.spatial.distance import cdist
        distances = cdist(peak_coords, peak_coords)
        
        # Find peaks that are too close to each other
        close_pairs = np.where((distances > 0) & (distances <= min_distance))
        
        # Create a set of peaks to keep (initially all)
        peaks_to_keep = set(range(len(peak_coords)))
        
        
        
        # Merge close peaks by keeping the one with highest value
        for i, j in zip(close_pairs[0], close_pairs[1]):
            if i < j and i in peaks_to_keep and j in peaks_to_keep:
                if peak_values[i] >= peak_values[j]:
                    peaks_to_keep.remove(j)
                else:
                    peaks_to_keep.remove(i)
        
        # Convert back to sorted list of indices
        peaks_to_keep = sorted(peaks_to_keep)
        
        # Get the merged peaks
        merged_rows = adjusted_rows[peaks_to_keep]
        merged_cols = adjusted_cols[peaks_to_keep]
        
        return np.stack((merged_rows, merged_cols), axis=1)
    else:
        return np.empty((0, 2), dtype=int)


Now we go to the hits and Hough accumulator

In [ ]:
def find_local_maxima_2d(hist_obj, threshold_abs=None, threshold_rel=0.1,
                         min_distance=5, smooth_sigma=1.5):
    """
    Find local maxima in a 2D ROOT histogram (TH2F/TH2D).
    Returns a list of (x, y) coordinates of peaks.
    """
    values, xedges, yedges = hist_obj.to_numpy()
    values = values.T
    values = np.nan_to_num(values, nan=0.0, posinf=0.0, neginf=0.0)

    # smooth histogram to suppress noise
    if smooth_sigma > 0:
        values = gaussian_filter(values, sigma=smooth_sigma)

    # find peaks (returns array of [y, x] indices)
    #coords = peak_local_max(values, threshold_abs=threshold_abs,
    #                        threshold_rel=threshold_rel,
    #                        min_distance=min_distance)
    coords = vectorized_2d_sliding_peaks(values, min_height=threshold_abs, window_size=2*min_distance)
    #print("coords ", type(coords), coords.shape)

    xcenters = 0.5 * (xedges[1:] + xedges[:-1])
    ycenters = 0.5 * (yedges[1:] + yedges[:-1])

    peaks = []
    for (y_idx, x_idx) in coords:
        peaks.append((xcenters[x_idx], ycenters[y_idx], values[y_idx, x_idx]))

    del xedges, yedges, xcenters, ycenters, coords
    gc.collect()

    return peaks, values

Get event and slice number

In [ ]:
def event_slice(text):
    # Split by common delimiters and check for exact match
    parts = text.replace('_', ';').split(';')
    
    return int(parts[1]), int(parts[2])

Check true slice

In [ ]:
# New easing function

import math
from dataclasses import dataclass

@dataclass
class HoughMeasurementStruct:
    cot: float
    vz: float

class HoughSlicer:
    def __init__(self, easing_type: str = "InSquare"):
        self.easing_type = easing_type
    
    def easing(self, x: float) -> float:
        """Easing function implementation"""
        if self.easing_type == "InSine":
            sign = 1 if x > 0 else (-1 if x < 0 else 0)
            return sign * 32 * (1 - math.cos((x * math.pi) / 64))
        elif self.easing_type == "InSquare":
            sign = 1 if x > 0 else (-1 if x < 0 else 0)
            return sign * 32 * (x * x / 1024)
        elif self.easing_type == "InCubic":
            return 32 * (x * x * x / 32768)
        elif self.easing_type == "InCirc":
            sign = 1 if x > 0 else (-1 if x < 0 else 0)
            return sign * (32 - math.sqrt(1024 - x * x))
        else:  # Linear
            return x
    
    def __call__(self, slice: int) -> (float, float):
        """Main slicer function"""
        if slice == -1:
            return -200 < meas.vz < 200
        else:
            lo_cot = self.easing(-32.0 + 2.0 * max(slice , 0) )
            hi_cot = self.easing(-32.0 + 2.0 * min(slice + 1 , 32) )  # we take true tracks from 1 neighbouring slice !!!!!
            #v1 = (meas.vz + 200) / meas.eta
            #v2 = (meas.vz - 200) / meas.eta
            #print(lo_cot, hi_cot)

            #return (meas.cot - lo_cot) * (meas.cot - hi_cot) < 0 and -200 < meas.vz < 200
            return lo_cot, hi_cot



Draw hough accumulator and true/reco peaks

In [ ]:
def draw_Hough(values, peaks, slice, true_peaks):
    
        #values = values0.copy()
        
        start_phi = 1000
        end_phi   = 2000
        size_true = 3

        
        # 2Create a figure and axes, then display the image
        fig, ax = plt.subplots(figsize=(16, 2*12))
        im = ax.imshow(values[start_phi:end_phi,:], cmap='viridis', interpolation='nearest')  
        # Adjust the aspect ratio to allow non-square cells
        ax = plt.gca()  # Get the current axes
        ax.set_aspect('auto')  # Allow non-square pixels

        plt.colorbar(im, label="Counts")  # Add color bar for values
        ax.set_xlabel("q/pT")  # Modify as per your data
        ax.set_ylabel("phi")  # Modify as per your data
        ax.set_title("Hough accumulator")  # Add a title

        # Patch a square around reco peaks
        for peak in peaks:
            # Define the square's properties
            square_x = peak[0] - size  # Bottom-left x-coordinate
            square_y = peak[1] - start_phi - size  # Bottom-left y-coordinate
            #print(square_x, square_y)

            # Create a Rectangle patch
            # The 'linewidth' defines the border thickness, 'edgecolor' the border color.
            # Use 'facecolor="none"' for a hollow square or remove it for a filled one.
            square = patches.Rectangle((square_x, square_y), 2*size, 2*size,
                                     linewidth=3, edgecolor='red', facecolor='none')

            # Add the square to the axes
            ax.add_patch(square)
            
        # Patch a small square around true peaks
        print(true_peaks.shape)
        
        # Get axis limits (image boundaries)
        xmin, xmax = ax.get_xlim()
        ymax, ymin = ax.get_ylim()
        for i in range(true_peaks.shape[0]): 
            # Define the square's properties
            square_x = int(true_peaks[i,1]) - size_true  # Bottom-left x-coordinate
            square_y = int(true_peaks[i,0]) - start_phi - size_true  # Bottom-left y-coordinate
            #print(square_x, square_y)

            # Create a Rectangle patch
            # The 'linewidth' defines the border thickness, 'edgecolor' the border color.
            # Use 'facecolor="none"' for a hollow square or remove it for a filled one.
            square = patches.Rectangle((square_x, square_y), 2*size_true, 2*size_true,
                                     linewidth=3, edgecolor="yellow", facecolor='none')

            
            ax.add_patch(square)
            
            # Draw number of hits directly on ax
            if xmin < square_x < xmax and ymin < square_y < ymax:
                ax.text(square_x, square_y, str(int(true_peaks[i, 6]))+" "+str(round(true_peaks[i,5],1)),
                        fontsize=10, color='white', fontweight='bold')

        plt.show()
        
        del values
        gc.collect

Save small squres (true and false)

In [ ]:
def get_Hough_squares(values, peaks, slice, true_peaks_with_mask, true_squares, false_squares):
    
    # true_peaks_with_mask is a tuple (true_peaks, mask)
    true_peaks, mask = true_peaks_with_mask
    
    # Process reconstructed peaks and extract squares
    # mas of the true peaks assigned to the reconstructed peaks
    true_peaks_mask = np.zeros(len(true_peaks), dtype=int)
    for peak in peaks:
        square_x = peak[0] - size
        square_y = peak[1] - size
        
        # Calculate center of the square
        center_x = square_x + size
        center_y = square_y + size
        
        # Create and add rectangle patch
        #square = patches.Rectangle((square_x, square_y), 2*size, 2*size,
        #                         linewidth=3, edgecolor='red', facecolor='none')
        #ax.add_patch(square)
        
        # Extract the square region from the values array
        x_start = max(0, int(square_y))
        x_end = min(values.shape[0], int(square_y + 2*size))
        y_start = max(0, int(square_x))
        y_end = min(values.shape[1], int(square_x + 2*size))
        
        # Copy the region to a new array
        if x_end > x_start and y_end > y_start:
            square_region = np.copy(values[x_start:x_end, y_start:y_end])
            
            # Check if there's a true peak within tolerance
            has_true_peak = False
            for i in range(true_peaks.shape[0]):
                true_x = true_peaks[i, 1]  # x coordinate of true peak
                true_y = true_peaks[i, 0]  # y coordinate 
                
                # Calculate distance from center of square to true peak
                distance = np.sqrt((center_x - true_x)**2 + (center_y - true_y)**2)
                # here the second condition means we want to have one to one assignment between reconstructed
                # peaks and true tracks 
                if distance <= tol and true_peaks_mask[i] < 0.5:
                    has_true_peak = True
                    true_peaks_mask[i] = 1
                    #break
            
            # Classify as true or false square
            if square_region.shape == (2*size, 2*size):
                if has_true_peak:
                    true_squares.append(square_region.astype(int))
                    # Change border color to green for true squares
                    #square.set_edgecolor('green')
                    #square.set_linewidth(4)
                else:
                    false_squares.append(square_region.astype(int))
                    # Keep red border for false squares
    
    # Get indices in the original true_peaks array
    mask_indices = np.where(mask)[0]
    
    # Map back to original array indices
    result_indices = mask_indices[true_peaks_mask.astype(bool)]
    
    # create an array of 0 and 1 
    result_mask = np.zeros(len(mask))
    result_mask[result_indices.astype(int)] = 1
    
    #print("mask_indices ", mask_indices, len(mask_indices))
    #print("true_peaks_mask", true_peaks_mask, len(true_peaks_mask))
    #print("result_indices", result_indices, len(result_indices))
    #print(result_mask)
    #print("result_mask = mask[true_peaks_mask] result_indices ",len(result_mask), len(mask), len(true_peaks_mask), len(result_indices))

    return true_squares, false_squares, result_mask

Match true and found peaks, write down small images around peaks

In [ ]:
def match_write(values, peaks, event, slice, truetracks, true_squares, false_squares, draw=True):
    
    true_peaks = np.squeeze(np.array(truetracks[event]))
    # Create slicer and check, whether the true track fits to the slice
    #print(true_peaks.shape)
    
    slicer = HoughSlicer(easing_type="InSquare")
    if slice>-1: # slice 0-32
        lo_cot, _ = slicer(0)
        _, hi_cot = slicer(32)
        #lo_cot, hi_cot = slicer(slice)
        mask = ( abs(true_peaks[:,3])<200 ) & ( lo_cot < true_peaks[:,5] ) & ( true_peaks[:,5] < hi_cot )
        #print("lo_cot, true_peaks[:,5], hi_cot ",lo_cot, true_peaks[:,5], hi_cot)
        #print(mask)
    else: # slice = -1
        lo_cot, _ = slicer(10)
        _, hi_cot = slicer(22)
        mask = (abs(true_peaks[:,3])<200 ) & ( lo_cot < true_peaks[:,5] ) & ( true_peaks[:,5] < hi_cot ) \
                & (true_peaks[:,1] > 0 + size) & (true_peaks[:,1] < nbin_qpt - size)
    
    #print(true_peaks[mask].shape)
    #print("true_peaks: ",true_peaks[mask])
    #print("reco peaks: ",peaks)
    
    # increase the n_truetracks
    global n_truetracks
    n_truetracks = n_truetracks + len(true_peaks[mask])
    
    
    if draw:
        draw_Hough(values, peaks, slice, true_peaks[mask])
    
    #get squares around peaks and indices of the associated true tracks
    true_squares, false_squares, result_mask = get_Hough_squares(values, peaks, slice, (true_peaks[mask], mask), \
                                                                    true_squares, false_squares)
    
    print("True tracks in event: ",len(true_peaks[mask])," True peaks total: ", len(true_squares))
    print("number of ones in result_mask ", np.count_nonzero(result_mask))
    
    # fill the "reco" column in truetracks marking the reconstructed true tracks (logical sum)
    if len(result_mask) == len(truetracks[event]):
        truetracks[event]['reco'] = result_mask.astype(int) | truetracks[event]['reco'].astype(int)
    else:
        print("Error! Length of truetracks[event]['reco'] and result_mask differ: ",len(truetracks[event]['reco']), len(result_mask))
    print("number of ones in reco ", np.count_nonzero(truetracks[event]['reco']))
    
    return true_squares, false_squares

In [ ]:
def process_root_file(file_path, truetracks, true_squares, false_squares, threshold_fraction=0.1, neighborhood_size=3):
    """
    Processes a single ROOT file, finds peaks in all 2D histograms,
    returns dict {hist_name: [(x, y), ...]}.
    """
    
    print("Processing file: ",file_path)
          
    results = {}
    ikey = 0
    

    
    with uproot.open(file_path) as f:
        for key in f.keys():  # iterate only over keys
            #if ikey>67:
            #    continue
            ikey +=1
            
            obj = f[key]      # load only one object at a time

            if not obj.classname.startswith("TH2"):
                continue
            #print(obj.name)  
                        
            #find true maxima
            event, slice = event_slice(key)
            
            #add event into the event list
            global event_list
            if event not in event_list:
                event_list.append(event)
                print("Event: ", event)
                
            
            #match true and found peaks
            if slice in slice_list:
                
                peaks, values = find_local_maxima_2d(obj,
                                     threshold_abs=threshold_abs,  # absolute threshold
                                     threshold_rel=threshold_rel,  # peaks > % of max
                                     min_distance=min_distance,    # at least n bins apart
                                     smooth_sigma=smooth_sigma)    # smooth before detection
                
                print("####### Slice number: ", slice)
                print("Peaks: ",len(peaks))
                true_squares, false_squares = match_write(values, peaks, event, slice, truetracks, 
                                                          true_squares, false_squares, ikey<200)

                del values
                
            # cleanup per histogram
            del obj
            gc.collect()


    
    gc.collect()
    return true_squares, false_squares

In [ ]:
# --- FIND ROOT FILES ---
root_files = sorted(Path(path).glob("out*.root"))

print(f"Found {len(root_files)} ROOT files in {path}\n")

# squares to be saved for NN training
true_squares = []
false_squares = []

# no of true tracks after selection
n_truetracks = 0

# list of events processed
event_list = []

# Create empty DataFrame
#mc_tracks  = pd.DataFrame(columns=['phi_bin', 'curv_bin', 
#                             'eta', 'vz', 'number_of_hits',
#                             'pz_over_pt', 'particle_type', 'reco'])


# --- LOOP THROUGH FILES AND LIST OBJECTS ---
ifile=0
for file in root_files:
    print("ifile = ",ifile)
    if ifile<num_files:
        true_squares, false_squares = process_root_file(file, truetracks, true_squares, false_squares)

    ifile += 1
    
           

In [ ]:
# Convert the list of arrays into a single numpy array of shape (1000, 28, 28)
array_true = np.stack(true_squares, axis=0)  # Stack along a new dimension
array_false = np.stack(false_squares, axis=0)  # Stack along a new dimension

#add column with class
y_true = np.ones(array_true.shape[0])
y_false = np.zeros(array_false.shape[0])
print("True and fake peaks: ",array_true.shape[0], ", ",array_false.shape[0])

print("Number of true tracks: ", n_truetracks)

# Combine arrays along the 0 axis
combined_array = np.vstack((array_true, array_false))
y_combined = np.hstack((y_true, y_false))

# Save the array to disk
np.savez('images', X = combined_array, y = y_combined)

print("Shapes of output arrays: ",combined_array.shape, y_combined.shape)


Convert dictionary of DataFrames to single DataFrame and save as ROOT ntuple

In [ ]:
def dict_to_root_ntuple(df_dict, event_list, filename, treename="ntuple", add_source_id=True):
    """
    Convert dictionary of DataFrames to single DataFrame and save as ROOT ntuple

    Parameters:
    df_dict: dictionary of {key: DataFrame}
    filename: output ROOT file name
    treename: name of the TTree
    add_source_id: whether to add a column identifying the source DataFrame
    """

    # Combine all DataFrames
    combined_dfs = []

    for key, df in df_dict.items():
        if key in event_list:
            df_copy = df.copy()
            #print(key, type(df_copy))
            # Add identifier column to track original source
            if add_source_id:
                df_copy['source_df'] = key

            combined_dfs.append(df_copy)

    # Concatenate all DataFrames
    combined_df = pd.concat(combined_dfs, ignore_index=True)

    print(f"Combined DataFrame shape: {combined_df.shape}")
    print(f"Columns: {combined_df.columns.tolist()}")

    # Prepare data for uproot: convert DataFrame columns to dictionary of NumPy arrays
    # Infer data types for mktree
    inferred_types = {col: combined_df[col].dtype for col in combined_df.columns}
    data_for_uproot = {col: combined_df[col].values for col in combined_df.columns}

    '''
    # Save as ROOT ntuple using uproot
    with uproot.recreate(filename) as file:
        file.mktree(treename, data_for_uproot, title=treename)

    print(f"Successfully saved to {filename}")
    return combined_df
    '''

    # Save as ROOT ntuple using uproot - SIMPLER APPROACH
    with uproot.recreate(filename) as file:
        file[treename] = combined_df.to_dict(orient='list')  # Convert to dict of lists
    
    print(f"Successfully saved to {filename}")
    return combined_df


Write true tracks as a ntuple

In [ ]:
#print(type(truetracks))
combined_truetracks = dict_to_root_ntuple(truetracks, event_list, "out_true_tracks.root", add_source_id=False)